Load libraries

In [13]:
require(ggplot2)
#require(GGally)
require(reshape2)
require(lme4)
require(compiler)
require(parallel)
require(boot)
# require(aod)
require(dplyr)

# Load data and set factors

In [2]:
mydata <- read.csv("C:\\Users\\Sarah\\Documents\\Personal Content\\Lab_study_data\\all_massaged_data\\dataframe_all_factors_for_analysis.txt",sep = '\t')
# # SID is the student number
mydata$sid <- factor(mydata$sid)
mydata$sim_index <- factor(mydata$sim_index)
mydata$lab_experience <- factor(mydata$lab_experience)
mydata$similar_sim <- factor(mydata$similar_sim)
mydata$CVS_graph <- factor(mydata$cvs_graph)
mydata$CVS_table <- factor(mydata$cvs_table)
# mydata$pre <- factor(mydata$pre)
# mydata$main <- factor(mydata$main)

In [3]:
head(mydata)
colnames(mydata)

sid,sim,variable,pre,main,cvs_graph,cvs_table,qual_score,quant_score,activity_order,...,CVS_context,use_table,use_graph,use_concentration,use_width,use_area,use_separation,use_all_vars,CVS_graph,CVS_table
10127163,L,Concentration,0,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,L,Width,0,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,C,Area,2,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10127163,C,Separation,2,2,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10232160,L,Concentration,0,0,1,1,1,1,LC,...,2,1,1,1,1,1,1,4,1,1
10232160,L,Width,0,0,0,0,1,1,LC,...,0,1,1,1,1,1,1,4,0,0


[1] "sid"                       "sim"                      
 [3] "variable"                  "pre"                      
 [5] "main"                      "cvs_graph"                
 [7] "cvs_table"                 "qual_score"               
 [9] "quant_score"               "activity_order"           
[11] "sim_index"                 "similar_sim"              
[13] "lab_experience"            "prior_number_virtual_labs"
[15] "overall.POCC"              "PC1"                      
[17] "PC2"                       "pre_with_ident"           
[19] "main_with_ident"           "CVS_context"              
[21] "use_table"                 "use_graph"                
[23] "use_concentration"         "use_width"                
[25] "use_area"                  "use_separation"           
[27] "use_all_vars"              "CVS_graph"                
[29] "CVS_table"

# Stat model 1: Predicting main model scores

We try to predict the type of model in the main worksheet (0,1 or 2 for neither, qual or quant). To do so, we have 10 independant variables:
* sim_index (1 or 2, wither it was student's 1st or 2nd activity)
* variable (thus don't include sim)
* CVS_graph (0 or 1)
* CVS_table (0 or 1)
* pre (0,1,2)
* student attributes: lab_experience, similar_sim, prior_number_virtual_labs, attitude components

We do so using a repeated measures anova where we have repeat measures of "variable" by student.

**Should we remove students with max pre for a certain variables???**

In [18]:
am1 <- aov(main
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am1)


Error: sid
                           Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_table                   1  21.10  21.101  31.621 1.01e-07 ***
cvs_graph                   1   2.83   2.825   4.234   0.0415 *  
pre                         1  12.57  12.571  18.839 2.74e-05 ***
lab_experience              1   0.23   0.233   0.350   0.5551    
similar_sim                 1   0.30   0.302   0.452   0.5024    
prior_number_virtual_labs   1   0.20   0.204   0.306   0.5809    
overall.POCC                1   0.34   0.342   0.513   0.4751    
PC1                         1   0.02   0.019   0.029   0.8658    
PC2                         1   0.58   0.584   0.874   0.3514    
Residuals                 137  91.42   0.667                     
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_table     1   2.84   2.841  12.470 0.000458 ***
cvs_graph     1   4.23   4.231  18.573 2.03e-05 ***
variable      3   

We see that the variance from having repeated measures for each student (first table) has been removed from the overall model error and thus separated in the output.

Looking the second table we notice that both cvs are significant, cvs_graph more significant than cvs_table, variable is significant and so is sim_index.

eta squares, and post hoc test are coming!

## Repeat to compare cvs_table to cvs_graph
We rerun the model for all students who did cvs_table to see if doing cvs_graph actually helps more.

In [9]:
am1_graph <- aov(main
    ~ cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=filter(mydata, cvs_table == 1))
summary(am1_graph)

Warning message:
"package 'bindrcpp' was built under R version 3.4.4"Warning message in aov(main ~ cvs_graph + variable + sim_index + pre + lab_experience + :
"Error() model is singular"


Error: sid
                           Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_graph                   1   6.27   6.267  11.616 0.000915 ***
variable                    3   2.54   0.847   1.571 0.200569    
sim_index                   1   1.53   1.526   2.828 0.095472 .  
pre                         1   5.38   5.377   9.966 0.002058 ** 
lab_experience              1   0.02   0.015   0.028 0.867067    
similar_sim                 1   0.07   0.073   0.135 0.713886    
prior_number_virtual_labs   1   0.01   0.007   0.014 0.906797    
overall.POCC                1   0.67   0.672   1.246 0.266760    
PC1                         1   0.28   0.276   0.511 0.476288    
PC2                         1   0.05   0.050   0.093 0.761382    
Residuals                 110  59.35   0.540                     
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value  Pr(>F)   
cvs_graph     1   2.19  2.1852   8.293 0.00436 **
variable

# Stat model 2: Predicting transfer data

In [5]:
am2 <- aov(quant_score
    ~  cvs_table + cvs_graph + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am2)


Error: sid
                           Df Sum Sq Mean Sq F value Pr(>F)  
cvs_table                   1   0.05  0.0542   0.130 0.7195  
cvs_graph                   1   2.04  2.0383   4.875 0.0289 *
pre                         1   1.58  1.5804   3.780 0.0539 .
lab_experience              1   0.06  0.0646   0.154 0.6949  
similar_sim                 1   0.62  0.6225   1.489 0.2245  
prior_number_virtual_labs   1   2.11  2.1087   5.044 0.0263 *
overall.POCC                1   0.19  0.1877   0.449 0.5040  
PC1                         1   0.05  0.0549   0.131 0.7177  
PC2                         1   0.43  0.4314   1.032 0.3115  
Residuals                 137  57.28  0.4181                 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value Pr(>F)  
cvs_table     1   0.06  0.0557   0.410 0.5222  
cvs_graph     1   0.24  0.2397   1.766 0.1845  
variable      3   1.35  0.4512   3.325 0.0197 *
sim_index     1   0.24  0.

As expected, CVS doesn't predict quant transfer scores.

Post hoc by variable and similar sim (weither students had worked with a similar sim before) coming soon.

In [6]:
am2 <- aov(quant_score
    ~  cvs_table + cvs_graph + main + variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + Error(sid/variable), data=mydata)
summary(am2)


Error: sid
                           Df Sum Sq Mean Sq F value Pr(>F)   
cvs_table                   1   0.05  0.0542   0.132 0.7167   
cvs_graph                   1   2.04  2.0383   4.977 0.0273 * 
main                        1   2.98  2.9769   7.269 0.0079 **
pre                         1   0.50  0.4975   1.215 0.2723   
lab_experience              1   0.10  0.1046   0.256 0.6140   
similar_sim                 1   0.51  0.5054   1.234 0.2686   
prior_number_virtual_labs   1   1.93  1.9331   4.720 0.0315 * 
overall.POCC                1   0.26  0.2623   0.641 0.4249   
PC1                         1   0.05  0.0464   0.113 0.7370   
PC2                         1   0.31  0.3074   0.751 0.3878   
Residuals                 136  55.70  0.4095                  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Error: sid:variable
             Df Sum Sq Mean Sq F value   Pr(>F)    
cvs_table     1   0.06   0.056   0.436   0.5096    
cvs_graph     1   0.24   0.240   1.875  

# Stat model 3: Predicting the use of CVS

## For cvs_table

In [7]:
# mydata$variable <- relevel(mydata$variable, "Width")
mixed <- glmer(
    cvs_table
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_table ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + overall.POCC + PC1 + PC2 + (1 |      sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   636.9    693.8   -305.4    610.9      575 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.7511 -0.4307  0.2088  0.3572  2.3885 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 5.991    2.448   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)   
(Intercept)               -1.20670    1.87139  -0.645  0.51905   
variableConcentration      0.53139    0.36369   1.461  0.14399   
variableSeparation        -0.25492    0.33945  -0.751  0.45267   
variableWidth              0.2

## For cvs_graph

In [8]:
mixed <- glmer(
    cvs_graph
    ~ variable + sim_index + pre
    + lab_experience + similar_sim + prior_number_virtual_labs + overall.POCC
    +  PC1 + PC2 + (1 | sid),
           data = mydata, family = binomial, 
           control = glmerControl(optimizer = "bobyqa"), nAGQ = 10)

summary(mixed)

Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 10) [glmerMod]
 Family: binomial  ( logit )
Formula: 
cvs_graph ~ variable + sim_index + pre + lab_experience + similar_sim +  
    prior_number_virtual_labs + overall.POCC + PC1 + PC2 + (1 |      sid)
   Data: mydata
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
   605.7    662.6   -289.9    579.7      575 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-2.0994 -0.3173 -0.1006  0.3446  2.8043 

Random effects:
 Groups Name        Variance Std.Dev.
 sid    (Intercept) 10.38    3.222   
Number of obs: 588, groups:  sid, 147

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                -6.0676     2.3957  -2.533  0.01132 *  
variableConcentration       0.7346     0.4053   1.812  0.06994 .  
variableSeparation         -0.1027     0.3733  -0.275  0.78319    
variableWidth             